# Density plots

In [18]:
library(raster)
library(RColorBrewer)
library(colorRamps)
library(viridis)
library(latex2exp)

Warning message:
"Paket 'latex2exp' wurde unter R Version 4.2.3 erstellt"


In [32]:
results <- read.csv("..\\data\\results.csv", header = TRUE, sep = ",")
results <- results[results$difference_data == "true", ]
results

,index,d,size,x,y,t,temp,comptype,burnin,iterations,duration,algorithm_statistic,action_data,energy_data,difference_data,correlation_data
,<int>,<int>,<int>,<int>,<int>,<int>,<dbl>,<chr>,<int>,<int>,<int>,<lgl>,<chr>,<chr>,<chr>,<chr>
5364,5363,3,4096,16,16,16,0.27,Metropolis Simulation,1000,409600,47,NA,false,false,true,false
5365,5364,3,46656,36,36,36,0.27,Metropolis Simulation,1000,409600,546,NA,false,false,true,false
5366,5365,3,46656,36,36,36,0.27,12x36 Wilson Metropolis Simulation,1000,409600,NA,NA,false,false,true,false
5367,5366,3,4096,16,16,16,0.27,5x16 Wilson Metropolis Simulation,1000,409600,NA,NA,false,false,true,false


## Time averaged field strength plots

In [25]:
plot_raster_field_strength <- function(max_x, max_y, values, name, charges, range) {
	require(raster)
	r <- raster(xmn = 0, xmx = max_x, ymn = 0, ymx = max_y, nrows = max_y, ncols = max_x)
	r[] <- values
	plot(r, col = plasma(256), xlab = "x", ylab = "y", xaxt = "n", yaxt = "n", zlim = range)
	title(TeX(paste("$", name, "$ field strength plot",
		ifelse(charges, " with Polyakov loops", ""), sep = ""), bold = TRUE))
	axis(1, at = seq(from = 0.5, by = 5, to = max_x), labels = seq(from = 0, by = 5, to = max_x))
	axis(2, at = seq(from = 0.5, by = 5, to = max_y), labels = seq(from = 0, by = 5, to = max_y))
	box(col = "white")
}

In [26]:
results <- read.csv("..\\data\\results.csv", header = TRUE, sep = ",")
results <- results[results$difference_data == "true", ]

entries <- data.frame(list(
	index = c(5363, 5366, 5364, 5365, 5363, 5366, 5364, 5365),
	direction = c(0, 0, 0, 0, 1, 1, 1, 1),
	blind = c(FALSE, FALSE, FALSE, FALSE, FALSE, FALSE, FALSE, FALSE),
	charges = c(FALSE, TRUE, FALSE, TRUE, FALSE, TRUE, FALSE, TRUE),
	#max_vals = c(0.0034, 0.25, 0.003, 0.25, 0.004, 0.125, 0.0032, 0.10005),
	#min_vals = c(-0.0035, -0.25, -0.003, -0.25, -0.004, -0.86, -0.003, -0.93)
	max_vals = c(0.25, 0.25, 0.25, 0.25, 0.125, 0.125, 0.10005, 0.10005),
	min_vals = c(-0.25, -0.25, -0.25, -0.25, -0.86, -0.86, -0.93, -0.93)
))

direction_names <- c("E_y", "E_x")

for (i in 1:nrow(entries)) {
	index <- entries$index[i]
	blind <- entries$blind[i]
	direction <- entries$direction[i]
	charges <- entries$charges[i]
	range <- c(entries$min_vals[i], entries$max_vals[i])

	max_x <- results$x[results$index == index]
	max_y <- results$y[results$index == index]

	d <- read.csv(paste("..\\data\\plot_data\\difference_", index, "_", direction, ".csv", sep = ""), header = TRUE, sep = ",")

	# Time averaging
	for (x in 0:(max_x - 1)) for (y in 0:(max_y - 1)) {
		d <- rbind(d, c(x, y, -1, mean(d$value[d$x == x & d$y == y])))
	}
	d <- d[d$t == -1, ]

	if (blind) {
		d$value[which.max(d$value)] <- 0
		d$value[which.min(d$value)] <- 0
		d$value[which.max(d$value)] <- 0
		d$value[which.min(d$value)] <- 0
	}

	d$x <- (d$x + floor(max_x / 3) + 1) %% max_x
	d$y <- (d$y + floor((max_y - 1) / 2)) %% max_y
	d <- d[order(d$y, d$x), ]

	png(filename = paste("plots\\field_strength_", index, "_", direction, ".png", sep = ""),
		width = 1440, height = 1440, type = c("cairo-png"), res = 250)
	plot_raster_field_strength(max_x, max_y, d$value, direction_names[direction + 1], charges, range)
	dev.off()
}

## Energy density plot



In [33]:
plot_raster_energy <- function(max_x, max_y, values, name, charges, range) {
r <- raster(xmn = 0, xmx = max_x, ymn = 0, ymx = max_y, nrows = max_y, ncols = max_x)
r[] <- values
plot(r, col = plasma(256), xlab = "x", ylab = "y", xaxt = "n", yaxt = "n", zlim = range)
title(paste("Energy density plot\n",
	ifelse(charges, " with Polyakov loops", ""), sep = ""))
axis(1, at = seq(from = 0.5, by = 5, to = max_x), labels = seq(from = 0, by = 5, to = max_x))
axis(2, at = seq(from = 0.5, by = 5, to = max_y), labels = seq(from = 0, by = 5, to = max_y))
box(col = "white")
}

In [35]:
results <- read.csv("..\\data\\results.csv", header = TRUE, sep = ",")
results <- results[results$difference_data == "true", ]

entries <- data.frame(list(
	index = c(5363, 5366, 5364, 5365),
	blind = c(FALSE, FALSE, FALSE, FALSE),
	charges = c(FALSE, TRUE, FALSE, TRUE),
	#max_vals = c(3.873752e-05, 3.711278e-01, 3.208702e-05, 4.342130e-01),
	#min_vals = c(-2.502818e-06,  1.002163e-05,  2.415023e-06,  2.464816e-06)
	max_vals = c(0.372, 0.372, 0.44, 0.44),
	min_vals = c(-0.001, 0, 0, 0)
))

max_val <- c()
min_val <- c()

for (i in c(1)) {
	index <- entries$index[i]
	blind <- entries$blind[i]
	charges <- entries$charges[i]
	range <- c(entries$min_vals[i], entries$max_vals[i])

	max_x <- results$x[results$index == index]
	max_y <- results$y[results$index == index]
	max_t <- results$t[results$index == index]

	dx <- read.csv(paste("..\\data\\plot_data\\difference_", index, "_", 0, ".csv", sep = ""), header = TRUE, sep = ",")
	dy <- read.csv(paste("..\\data\\plot_data\\difference_", index, "_", 1, ".csv", sep = ""), header = TRUE, sep = ",")
	dt <- read.csv(paste("..\\data\\plot_data\\difference_", index, "_", 2, ".csv", sep = ""), header = TRUE, sep = ",")

	dx$value <- dx$value * dx$value
	dy$value <- dy$value * dy$value
	dt$value <- dt$value * dt$value

	d <- dx
	d$value <- 0
	for (i in 1:nrow(d)) {
		x <- d$x[i]
		y <- d$y[i]
		t <- d$t[i]
		d$value[i] <- dx$value[dx$x == x & dx$y == y & dx$t == t] +
			dy$value[dy$x == x & dy$y == y & dy$t == t] -
			dt$value[dx$x == x & dt$y == y & dt$t == t] +
			dx$value[dx$x == (x - 1) %% max_x & dx$y == y & dx$t == t] +
			dy$value[dy$x == x & dy$y == (y - 1) %% max_y & dy$t == t] -
			dt$value[dx$x == x & dt$y == y & dt$t == (t - 1) %% max_t]
	}

	d$value <- d$value / 2

	# Time averaging
	for (x in 0:(max_x - 1)) for (y in 0:(max_y - 1)) {
		d <- rbind(d, c(x, y, -1, mean(d$value[d$x == x & d$y == y])))
	}
	d <- d[d$t == -1, ]

	d$x <- (d$x + floor(max_x / 3) + 1) %% max_x
	d$y <- (d$y + floor((max_y - 1) / 2)) %% max_y
	d <- d[order(d$y, d$x), ]

	
	png(filename = paste("plots\\energy_density_", index, "_", direction, ".png", sep = ""),
		width = 1440, height = 1440, type = c("cairo-png"), res = 250)
	plot_raster_energy(max_x, max_y, d$value, direction_names[direction + 1], charges, range)
	dev.off()

	max_val <- append(max_val, max(d$value))
	min_val <- append(min_val, min(d$value))
}